# RNAseq in a nutshell: From FASTQ files to differential expression
## Part II

Last session, we worked through how to take sequencing reads derived from RNA-Seq, stored as `fastq` files, into transcripts and gene counts. For this, we used `Salmon`, but keep in mind that this tools is one of a collection of different methods which can do that. 

In this session, we use the `DESeq2` package to perform some exploratory analysis, examining the relationship between the samples and then calculate differential expression between different sample attributes; e.g., treatment vs. non-treatment, or the source from which the sample was obtained.

RNA-Seq pipelines are composed of several steps, which includes:

- sample normalization
- filtering
- variance transformation
- and statistical modeling to determine differential expression

First, let’s load the data and required packages, we have already provided you with a gene level SummarizedExperiment object that contains the full airways dataset.

**Q1.** Execute the code below to load the `DESeq2` package, and to load the object `gse` from last time into your workspace.

In [3]:
#library('DESeq2')
#library('tidyverse')
load('../08_Data_RNASeq-II/gse.RData')

As a reminder, `gse` is a variable which stores a **SummarizedExperiment** object, this is a special object used that is designed to store results of RNA-Seq experiments. In this case, `gse` stores the output of running the Salmon pipeline. 

The results of many other mapping algorithms (e.g. `Kalisko`, `RSEM`, `STAR`, etc.) can also be loaded into a **SummarizedExperiment** object, which can then be used as input to additional differential expression analysis pipelines other than `DESeq2` (e.g., `EdgeR`).

**Q2.** Let's examine the `gse` object. 

- use `head()` to get a summary of the data stored in `gse`
- use `colData()` to look at the metadata associated with each sample in `gse`.

**Provide and Execute your code below.**

When running the `colData` command, you will notice that the data type for sample attributes is `factor`. This is a very convenient data type used in R to define categorical variables. This is necessary when a particular set of features does not come from a natural ordering (i.e., red and green) and/or when one has more than one "level" (i.e., red, green, and blue). Factors have predefined levels that can be defined and arranged in different ways.

Factors have a range of operations that can be performed on them. e.g., you can obtain the list of 'levels' for your a variable that is a factor via:

    levels(var_isafactor)

**Q3.** Rename the label of `gse` factor `condition` to 'untrt' and 'trt', accordingly.

- Use dollar-sign `$` notation to obtain the list of `condition` in the variable `gse`
- Use `levels()` on this list to obtain the set of levels for the list 
- Reassign labels with `untrt` and `trt` for the untreated and Dexamethasone treatment (Hint: `c()`)
- Use `colData()` to confirm the condition has been relablled.

(Note: This might seem familiar to you when using `rownames()` or `colnames()` to rename rows/columns. The idea is the similar here.)

**Normalization:**<br>
Normalization, in general, is important so that genes which may be 'different' from one another - that may be heterogeneous in read depth are cross comparable. The number of read counts detected for each gene will need to be normalized for three major factors:<br>
* Library size – unequal mixing of samples for sequencing, or sequencing samples on different high-thoughput sequencing runs will result in differences in the total number reads called for each sample (this is commonly referred to as sequencing depth).
* Gene length – the longer the gene the more likely we are to identify RNA fragments for each it.
* Library composition – initial normalization methods did not correct for this. Normalization methods that take this into account were only introduced after `DESeq2`. We will discuss the problem of library composition and how the `DESeq2` pipeline accounts for it below.

Let’s compare the total number or reads between samples. Recall that our object `se` contains several assays: 

`counts` with the raw counts for each gene

`abundance` with normalized counts (i.e., transcripts per Million mapped reads, or TPM) and 

`length` containing the estimated length for each gene/transcript. 

**Q4.** Provide code that summarizes the total `counts` (in millions) across samples. To do that:

- Use dollar sign `$` notation and `assays()` to obtain `counts` stored in `gse`.
- Provide this to the function `colSums()` to obtain the total reads per sequencing experiment
- rescale to units of 1e6 (i.e., 1 million)

**Provide and Execute your code below.**

**Q5.** Provide code that summarizes the total `abundance` across samples. To do that:

- Use dollar sign `$` notation and `assays()` to obtain `abundance` stored in `gse`.
- Provide this to the function `colSums` to obtain the total reads per sequencing experiment

**Provide and Execute your code below.**

The `abundance` contains TPM values, counts which are normalized by gene length and library size, which is why the sum of the columns in the `abundance` assay results in the same amount. Previous normalization methods, like RPKM (Reads Per Kilobase of transcript, per Million) were very similar to TPM, only that they normalized first for library size and then for gene length, resulting in slight differences in the total number of counts per sample. TPM is considered a slightly improved metric. Some of these terms: RPKM, FPKM, and TPM can be confusing, but [this video](https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/) gives a prettly clear explanation of the differences if you want to know more.

*Library composition:*<br>
However, TPM - the generally preferred approach to normalize for depth and transcirpt length - does **not** take into account library composition. 

To understand this problem, let's consider the following example. 

Imagine you have two groups (`A` and `B`), and consider four genes that are the same length. 

Let's say `gene1` is truly upregulated (2.5x) from treatment (`groupB`), and that (magically) we also know that `gene2`, `gene3`, and `gene4` expression truly does not change between groups.

Now let's say you sequenced and obtained *exactly* 2000 reads from `groupA` and `groupB`. You might reasonably expect to see the following distribution of counts:

           groupA   groupB
    gene1     500     1250   
    gene2     500      250 
    gene3     500      250
    gene4     500      250

If you took these reads at face and tested each gene for differential expression: all of these genes might be reported as differentially expressed, instead of just `gene1`. The size of genes are the same (so length normalization won't fix this) and we've sequenced *exact* 2000 reads in both groups, so normalizing for overall sequencing depth won't adjust this either. Why is this?

The key idea here is that reads are a *finite resource*. The more reads that `gene1` accrues, the less reads (and counts) in other genes. Sequencing counts are a *relative* measurement, not an absolute one. This is in contrast to measurement intensities from microarray data we previously worked with (at least, they are *approximately* absolute).

Thus, we should have a normalization strategy that aims to account for this. The `DESeq2` pipeline addresses this issue by using a *median ratio normalization*. We will not get into the details here, but just remember that scaling factors calculated as part of running the full `DESeq2` pipeline below are a result of such normalization.

**Q6.** To start using the `DESeq2` pipeline, use the code below to construct a `DESeq2` object from the **SummarizedExperiment** object. 

The parameter design specifies the comparisons that `DESeq2` will perform later when we start comparing samples. Note that the initial `dds` object, generated by running the code below, has a similar structure to a **SummarizedExperiment** object.

In [ ]:
dds <- DESeqDataSet(gse, design = ~ donor + condition)
head(dds)

When building the `DESeq2` object - just like we did in with the microarrays in the previous modules - we need define the experimental *design*. Here, we have used `~ donor + condition`. This will tell the algorithm which comparisons to make.

**Filtering:**<br>
Our current dataset contains many rows (genes) that are covered by very few read counts. It is always a good idea to filter out non-informative rows and reduce the size of our dataset.

**Q7.** Use the code below to filter out all genes that have less than 10 read counts across samples. How many genes have been filtered out?

In [ ]:
nrow(dds)
keep <- rowSums(counts(dds)) > 10
dds <- dds[keep,]
nrow(dds)
## The number of genes filtered with few reads counts is: 

**Variance stabilizing transformations:**<br>
In most RNA-Seq data sets, we will observe a relationship between the mean and the variance of read counts, whereby the variance will be higher for more lowly-expressed genes. 

This effect is due to sampling: the higher the expression level of a gene is, the more reads we will identify for it; but the more reads we accrue, the less *uncertainty* (variance) we have about the gene expression levels (i.e., recall that the error on the mean expression is proportional to the  size (in this case, read counts) we sample).  

**Q8.** To examine this problem, use the code below to plot the log2(TPM) values between two samples.

In [ ]:
tpm_mat <- as.data.frame(assays(dds)$abundance)
ggplot(data=tpm_mat,aes(x=SRR1039508,y=SRR1039509)) + geom_point(size=0.5) + scale_x_continuous(trans = 'log2') + scale_y_continuous(trans = 'log2')

The increase in variance as genes are expressed less introduces two main artifacts:<br>

(1) It will be harder to identify *significant* differentially expressed genes at lower expression levels. <br> 

(2) When comparing samples to each other using the whole transcriptome, lowly expressed genes will have more impact on these comparisons.

`DESeq2` offers two methods for variance stabilization that can be run outside of the main `DESeq2` command (which we will run below):
* VST - variance stabilizing transformation 
* Rlog - regularized-logarithm transformation

Both methods will have a similar effect to a standard log2 transform for high expression values but will have a more dramatic effect on the low expressing genes by "shrinking" the values towards the null (i.e., no difference between samples).

**Q9.** Execute the code below to generate two new objects containing transformed values using these two methods.

In [ ]:
vsd <- vst(dds, blind = FALSE)
head(vsd)
rld <- rlog(dds, blind = FALSE)
head(rld)

We use the flag `blind = FALSE` to make sure that sample labels are taken into account when doing this transfomations. Note that when looking at the `vsd` and `rld` objects, we will only see one assay containing the transformed values. Those values can be accessed by simple `assay(vsd)` or `assay(rld)`.

Let’s use our tidyverse skills to examine the effect of these transformations!

**Q10.** Complete the below code to construct a new *tidy* data frame with values from the non-transformed TPM values and from the objects from the two data transformations in **Q9**.

In [ ]:
count_mat <- bind_rows(as_tibble(log2(assays(dds)$abundance)) %>% mutate(transformation = "none"),
                       as_tibble(assay(vsd)) %>% mutate(transformation = "vsd"),
                       ________________________________________________________)

**Q11.** Recreate the plot you made in **Q8** with these data

- use `count_mat`, remove the log scaling of the axis (note that `vsd` and `rld` already transforms the data to a logarithmic scale)
- use the `transformation` column as your facet layer in ggplot.

Note how both methods reduced the inflated fold changes associated with low expression values.

**Examining the similarity between samples using dimensionality reduction:**<br>
After normalization, we next want to explore the relationship between samples by comparing whole transcriptome profiles. We do this to check that replicate samples are similar to each other and that there are no obvious and major batch effects (i.e., we expect all samples processed on the same day and in the same sequencing pool to have similar profiles of expression and cluster together) and to identify the parameters that are responsible for the most noted difference between samples. 

There are two general ways to do this: using clustering techniques or dimensionality reduction. We will cover dimensionality reduction in more detail during the single cell RNA-Seq part of this course. Here, we will use [Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis), which is a popular and simple method for dimensionality reduction. 

To give an intuition around what PCA does and how it works: Imagine each of our samples that we quantified expression of only 3 genes. In this case, we could make a 3D plot using each gene expression level as an axis. In such a plot, samples with similar expression profiles would appear more closely together. In contrast, samples whose expression profile changed will be separated. 

In practice, our samples contain expression information about almost 20,000 genes! So we cannot plot each gene on a separate axis. To address this, PCA uses a simple mathematical transformation to project the data of all 20,000 genes onto a much smaller number of dimensions, from which we can select 2-3 to plot and visualize. Briefly, PCA iteratively calculates principal components (PC), which are transformations of the whole data matrix onto the plot axis, while trying to preserve a maximum amount of the variability in the data. Each PC describes some amount of variability it captures across all genes. (In mathematical terms, principal components are simply the eigenvectors of the data's covariance matrix).

**Q12.** Use the code below to run a PCA on your rlog normalized samples. There are several PCA functions in R. Here, we will use one that part of the `DESeq2` package. 

- The parameter `intgroup` specifies which sample attributes to use on the legend (or, looks like this PCA function uses ggplot! So `intgroup` defines addtional aesthetic mappings associated with your samples). 

In [ ]:
plotPCA(rld, intgroup = c("condition", "donor"))

**Q13.** What is the sample attribute that is responsible to the largest variance between samples? (Hint: look at the principal component that captures the most of the variance). 

**Q14.** In the legend, you can see a set of sample identifiers attached to the treatment conditions. Do all of the sample seem to cluster similarly and behave the same? Why or why not?

**Running the full DESeq2 pipeline:**<br>
Now *finally* you are in a position to run the full `DESeq2` pipeline to look for differentially expressed genes that underlie the differences between samples that we observed in our PCA plot.

This is done in a single line of code. **Execute it below!**

In [ ]:
dds <- DESeq(dds)

Although we have tested different data transformations above, when running the pipeline, we always start from non-normalized counts. Running the code above has performed these steps:

* Median ratio normalization to correct for library size and composition.
* Estimating dispersion
* Modeling the data using a negative binomial model to calculate gene-based p-values

Note that here, `DESeq()` handles the variance stabilization issue, rather than by calculating vst or rlog forms, instead by modeling this dispersion statistically (via the negative binomial model). The Vst and rlog forms are generally useful for visualization and other applications.

We can look at the results for each one of the comparisons we defined above when constructing the initial DESeq object (remember `design =`?) using the code below.

**Execute the code below.**

In [ ]:
res <- results(dds,contrast=c("condition","trt","untrt"),tidy=T)
res <- res %>% as_tibble() %>% separate(row,into=c("ensembl_gene_id","tx_id_num"), sep="[.]")
head(res)

**Q15.** Next:

- Sort the object `res_annot` by `padj`

What are the top five hits from this differential expression experiment?

**Provide and Execute your code below.**

In [ ]:


## The top five hits are: 

**Q16.** Note that the default multiple test corection (padj) uses the Benjamin-Hochberg correction procedure (i.e., `p.adjust(method='BH')`). `results()` is actually a fairly flexible function and can be used to summarize your results in many ways using various arguments provided to the function. You can learn more about `results()` [here](https://www.rdocumentation.org/packages/DESeq2/versions/1.12.3/topics/results). 

You can also couple `summary()` together via: `summary(results())` to obtain compact summaries of those results!

Using `results()` and `summary()` to report how many genes pass significance assessed:

- an adjusted p-value < 0.05
- an adjusted p-value < 0.05 **and** log2-Fold change > 0.585 or < -0.585

(Note: the default output is log2FoldChange)

**Provide and Execute your code below.**

**Q17.** Now go back to the PCA plot you produced above in **Q12**. There seemed to be one donor that appeared to be an outlier. Compare that donor to the sample "N052611". 

- modify the following code to create a new 'design' matrix for analysis:

        results(dds,contrast=c("_____","_______","_______"),tidy=T)


(Hint: read about what the 'contrast' argument is for in `results()`.)

- `arrange()` this table by `padj`.
- Report the top hit.

**Provide and Execute your code below.**